# Imports

In [2]:
from notebooks.data_collection import *
from notebooks.data_cleaning import *

ModuleNotFoundError: No module named 'notebooks'

# Collecting Data

In [2]:
collect_data()

The file '../ift6758/data/data_saison_2016_2017_play_by_play.json' already exists.
The file '../ift6758/data/data_saison_2017_2018_play_by_play.json' already exists.
The file '../ift6758/data/data_saison_2018_2019_play_by_play.json' already exists.
The file '../ift6758/data/data_saison_2019_2020_play_by_play.json' already exists.
The file '../ift6758/data/data_saison_2020_2021_play_by_play.json' already exists.
The file '../ift6758/data/data_total_play_by_play.json' already exists.


# Cleaning Data

In [3]:
json_path = "../ift6758/data/data_total_play_by_play.json"

data = pd.read_json(json_path)

create_event_df(data)
create_faceoff_df(data)
create_player_df(data)
create_shoot_df(data)
create_takeaway_df(data)

The file ../ift6758/data_clean/eventdf_clean.csv already exists.
The file ../ift6758/data_clean/faceoffdf_clean.csv already exists.
The file ../ift6758/data_clean/playerdf_clean.csv already exists.
The file ../ift6758/data_clean/shootdf_clean.csv already exists.
The file ../ift6758/data_clean/takeawaydf_clean.csv already exists.
